# GOES-R Lightning Data: Max Flash Point Extraction

This notebook processes GOES-R Lightning NetCDF datasets to extract the maximum lightning flash point from each file. It converts the satellite-projected coordinates to geographic coordinates (latitude and longitude), aggregates the maximum flash locations, and exports the results as a GeoJSON file for visualization and further analysis.

The data used in this notebook comes from the **GOES-R Series Geostationary Lightning Mapper (GLM)** Level 2 NetCDF products provided by NOAA/NESDIS. These datasets capture lightning flash information across the Americas with high spatial and temporal resolution.

For more information, visit the official NOAA GLM product page:  
https://www.nesdis.noaa.gov/GOES-R-series/geostationary-lightning-mapper


---

## How to Retrieve GOES-R Lightning Data from NASA Earthdata

You can download GOES-R GLM data from NASA's Earthdata Search portal:

🔗 https://search.earthdata.nasa.gov/

### Steps:
1. Go to [search.earthdata.nasa.gov](https://search.earthdata.nasa.gov/).
2. In the search bar, enter: **"GLM L2 Lightning Detection GOES"** or a more specific product name like `"OR_GLM-L2-LCFA_G16"` or `"GLMF"`.
3. Use the map to narrow the spatial region (if desired).
4. Set your date range in the filter panel.
5. Browse the results and click **Download All** or use the **Customize** button to select specific files.
6. You’ll need a free [Earthdata Login](https://urs.earthdata.nasa.gov/users/new) to access and download files.

The downloaded `.nc` files can then be processed using the script in this notebook.

---

## Script Overview

- Opens GOES-R NetCDF files from a specified folder.
- Identifies the pixel with the maximum total optical energy (a proxy for lightning intensity).
- Converts pixel coordinates from geostationary projection (radians) to latitude/longitude using the correct satellite longitude and geostationary CRS.
- Compiles all maximum flash points into a GeoDataFrame.
- Exports the combined points as a GeoJSON file for easy use in GIS software and web mapping.


---

**Michael Huff**  
GIS Analyst & Developer

*Feel free to reach out for questions or collaboration!*

michaelhuff17@gmail.com


In [ ]:
import os
import xarray
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS, Transformer
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

# -------------------------
# CONFIGURATION
# -------------------------
input_folder = r"E:\GOES-R Lightning Data\Test Batch"
output_file = r"E:\GOES-R Lightning Data\Processed\test_batch_max_optical_flashes.geojson"
variable_name = "Total_Optical_energy"
sat_height = 35786023.0
wgs84 = CRS.from_epsg(4326)

# -------------------------
# PRE-COMPUTED TRANSFORMERS
# -------------------------
def build_transformer(sat_lon):
    proj_str = (
        f"+proj=geos +h={sat_height} +lon_0={sat_lon} "
        "+sweep=x +a=6378137 +b=6356752.31414 +units=m +no_defs"
    )
    return Transformer.from_crs(CRS.from_proj4(proj_str), wgs84, always_xy=True)

transformers = {
    "east": build_transformer(-75.0),
    "west": build_transformer(-137.0),
    "default": build_transformer(-75.0),
}

# -------------------------
# FILE PROCESSING FUNCTION
# -------------------------
def process_file(file):
    if not file.endswith(".nc"):
        return None

    filepath = os.path.join(input_folder, file)
    try:
        with xarray.open_dataset(filepath) as ds:
            if variable_name not in ds:
                return None

            slot = ds.attrs.get('orbital_slot', '').lower()
            key = "east" if "east" in slot else "west" if "west" in slot else "east"
            transformer = transformers[key]

            energy_var = ds[variable_name]
            energy_raw = energy_var.values.astype(float)

            # Mask fill values
            fill_value = energy_var.attrs.get('_FillValue', 0)
            energy_raw[energy_raw == fill_value] = np.nan
            energy = energy_raw

            if np.count_nonzero(~np.isnan(energy)) == 0:
                return None

            max_index = np.unravel_index(np.nanargmax(energy), energy.shape)
            max_value = energy[max_index]
            if np.isnan(max_value) or max_value <= 0:
                return None

            x = ds['x'].values.astype(float)
            y = ds['y'].values.astype(float)

            x_rad = x[max_index[1]]
            y_rad = y[max_index[0]]
            x_m = x_rad * sat_height
            y_m = y_rad * sat_height
            lon, lat = transformer.transform(x_m, y_m)

            return {
                'filename': file,
                'energy': max_value,
                'geometry': Point(lon, lat)
            }

    except Exception as e:
        print(f"Error processing {file}: {e}")
        return None

# -------------------------
# MAIN EXECUTION
# -------------------------
if __name__ == "__main__":
    file_list = [f for f in os.listdir(input_folder) if f.endswith(".nc")]
    print(f"Processing {len(file_list)} files...")

    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        results = list(executor.map(process_file, file_list))

    records = [r for r in results if r is not None]

    if records:
        gdf = gpd.GeoDataFrame(records, crs="EPSG:4326")
        gdf.to_file(output_file, driver="GeoJSON")
        print(f"Exported {len(records)} points to {output_file}")
    else:
        print("No valid data extracted.")
